## References
https://www.medi-08-data-06.work/entry/regression_stan

In [1]:
import numpy as np
import numpy.random as rd
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import animation as ani
import matplotlib.cm as cm
#import seaborn as sns
#sns.set(style="whitegrid", palette="muted", color_codes=True)

#from tabulate import tabulate
from time import time

import pystan
from pystan import StanModel

import statsmodels.api as sm
import statsmodels.formula.api as smf

### data

In [2]:
data = pd.read_csv("./data.csv")
col_name = data.columns
data

,Region,Type,1st Size,2nd Size,1st mean,2nd mean,1st std,2nd std
0,A,1,55,51,151.36,157.27,2.94,2.98
1,B,1,53,49,151.56,156.83,3.07,3.14
2,C,0,55,53,152.22,157.08,3.20,3.21
3,D,1,53,52,153.09,156.00,2.65,2.64
4,E,1,58,55,153.22,157.24,3.07,3.03
5,F,0,55,53,153.31,157.22,3.10,3.13
6,G,0,58,53,152.98,157.81,2.49,2.45
7,H,0,59,57,153.27,158.95,3.08,3.06
8,I,1,56,51,152.67,156.82,2.82,2.92
9,J,0,56,50,155.37,161.71,3.10,3.21


In [3]:
# data preparation
Y = np.hstack([data["1st mean"], data["2nd mean"]])
Age = np.hstack([np.zeros(10), np.ones(10)])
X = np.hstack([data["Type"], data["Type"]])
X = np.hstack([X.reshape(-1,1), Age.reshape(-1,1)])
X = pd.DataFrame(X)

### model1

In [4]:
# data preparationfor model1
data_model1 = pd.concat([X,pd.DataFrame(Y)], axis=1)
data_model1.columns = ["Type", "Age", "meanY"]
data_model1

,Type,Age,meanY
0,1.0,0.0,151.36
1,1.0,0.0,151.56
2,0.0,0.0,152.22
3,1.0,0.0,153.09
4,1.0,0.0,153.22
5,0.0,0.0,153.31
6,0.0,0.0,152.98
7,0.0,0.0,153.27
8,1.0,0.0,152.67
9,0.0,0.0,155.37


In [5]:
formula = "meanY ~  1 + Age + Age:Type"
link = sm.genmod.families.links.identity
family = sm.families.Gaussian(link=link)

mod = smf.glm(formula=formula, data=data_model1, family=family )
result = mod.fit() 
print(result.summary())

print(result.aic)

                 Generalized Linear Model Regression Results                  
Dep. Variable:                  meanY   No. Observations:                   20
Model:                            GLM   Df Residuals:                       17
Model Family:                Gaussian   Df Model:                            2
Link Function:               identity   Scale:                          1.5827
Method:                          IRLS   Log-Likelihood:                -31.345
Date:                Tue, 23 Jun 2020   Deviance:                       26.907
Time:                        21:21:04   Pearson chi2:                     26.9
No. Iterations:                     3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    152.9050      0.398    384.341      0.0

C:\Users\chiak\Anaconda3\envs\stan\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Calling Family(..) with a link class as argument is deprecated.
Use an instance of a link class instead.
  This is separate from the ipykernel package so we can avoid doing imports until


### Bayes model1

In [6]:
data_model1.iloc[:,:2]

,Type,Age
0,1.0,0.0
1,1.0,0.0
2,0.0,0.0
3,1.0,0.0
4,1.0,0.0
5,0.0,0.0
6,0.0,0.0
7,0.0,0.0
8,1.0,0.0
9,0.0,0.0


In [7]:
X = data_model1.iloc[:,:2]
y = data_model1.iloc[:,-1]

In [8]:
dat = {'N': X.shape[0], 'M': X.shape[1], 'X': X, 'y': y}

In [9]:
stan_model = """
data {
    int<lower=0> N;
    int<lower=0> M;
    matrix[N,M] X;
    real y[N];
}

parameters {
    real beta1;
    real beta2;
    real beta3;
    real<lower=0> sigma;
}

model{
    for (n in 1:N){
      y[n] ~ normal(beta1 + beta2 * X[n,2] + beta3*X[n,1]*X[n,2], sigma);
    }
    beta1 ~ uniform(-1e+4, 1e+4);
    beta2 ~ uniform(-1e+4, 1e+4);
    beta3 ~ uniform(-1e+4, 1e+4);
    sigma ~ uniform(-1e+4, 1e+4);
}
 
"""

In [10]:
# Build Stan model
%time stm = StanModel(model_code=stan_model)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_433ba1d713981081b1bc4b0f8c03d857 NOW.


Wall time: 2min 5s


In [11]:
n_itr = 3000
n_warmup = 200
chains = 2

# サンプリングの実行
fit = stm.sampling(data=dat, iter=n_itr, chains=chains, n_jobs=-1, warmup=n_warmup, algorithm="NUTS", verbose=False)
la    = fit.extract(permuted=True)  # return a dictionary of arrays
# パラメーター名
names = fit.model_pars 
# パラメーターの数
n_param = np.sum([1 if len(x) == 0 else x[0] for x in fit.par_dims])
print(fit)

Inference for Stan model: anon_model_433ba1d713981081b1bc4b0f8c03d857.
2 chains, each with iter=3000; warmup=200; thin=1; 
post-warmup draws per chain=2800, total post-warmup draws=5600.

        mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
beta1 152.91  8.4e-3   0.44 152.03 152.62 152.91 153.19 153.81   2801    1.0
beta2   5.65    0.02   0.76   4.13   5.16   5.67   6.14   7.17   2463    1.0
beta3  -1.73    0.02   0.87  -3.48  -2.29  -1.72  -1.16 6.4e-4   3245    1.0
sigma   1.36  4.6e-3   0.26   0.95   1.17   1.32   1.51   1.99   3335    1.0
lp__  -15.09    0.04   1.61 -19.19 -15.91 -14.71  -13.9 -13.08   1792    1.0

Samples were drawn using NUTS at Tue Jun 23 21:23:18 2020.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
convergence, Rhat=1).


### model 2

In [39]:
Y = np.vstack([data["1st mean"], data["2nd mean"]])
X = np.vstack([data["Type"], data["Type"]])
Age = np.vstack([np.zeros(10),np.ones(10)])
dat = {'T': X.shape[0], 'P':X.shape[1], 'Age':Age, 'X': X, 'y': y}

In [95]:
stan_model2 = """
data {

    int<lower=0> T;
    int<lower=0> P;
    matrix[T,P] X;
    matrix[T,P] Age;
    matrix[T,P] y;
    
}

parameters {

    real beta1;
    real beta2;
    real beta3;
    real<lower=0> sigma;
    matrix[T,P] mu;
    matrix[T,P] r;
    vector[T] s;
    
}

model{
    for (i in 1:T){
        for (j in 1:P){
             y[i,j] ~ normal(mu[i,j], sigma);
             
             mu[i,j] = beta1 + r[1,j] + (beta2 + beta3*X[i,j] + r[2,j])*Age[i,j];
             
             }
     }
     
    beta1 ~ uniform(-1e+4, 1e+4);
    beta2 ~ uniform(-1e+4, 1e+4);
    beta3 ~ uniform(-1e+4, 1e+4);
    sigma ~ uniform(-1e+4, 1e+4);
    for (i in 1:T){
        for (j in 1:P){
            r[i,j] ~ normal(0,s[i]);
         }
         s[i] ~ uniform(0, 1e+4);
        }
}
 
"""

In [92]:
stan_model2 = """
data {

    int<lower=0> T;
    int<lower=0> P;
    matrix[T,P] X;
    matrix[T,P] Age;
    matrix[T,P] y;
    
}

parameters {

    real beta1;
    real beta2;
    real beta3;
    real<lower=0> sigma;
    
}

transformed parameters {
    matrix[T,P] mu;
    matrix[T,P] r;
    vector[T] s;
    
    for (i in 1:T){
        for (j in 1:P){
            mu[i,j] = beta1 + r[1,j] + (beta2 + beta3*X[i,j] + r[2,j])*Age[i,j];
        }
    }
    for (i in 1:T){
        for (j in 1:P){
            r[i,j] ~ normal(0,s[i]);
         }
         s[i] ~ uniform(0, 1e+4);
        }
}

model{
    for (i in 1:T){
        for (j in 1:P){
             y[i,j] ~ normal(mu[i,j], sigma);
             }
     }
     
    beta1 ~ uniform(-1e+4, 1e+4);
    beta2 ~ uniform(-1e+4, 1e+4);
    beta3 ~ uniform(-1e+4, 1e+4);
    sigma ~ uniform(-1e+4, 1e+4);
    
}
 
"""

In [96]:
# Build Stan model
%time stm = StanModel(model_code=stan_model2)

ValueError: Failed to parse Stan model 'anon_model_46232ef7d40f84269b6a431261bcb40f'. Error message:
SYNTAX ERROR, MESSAGE(S) FROM PARSER:
Cannot assign to variable outside of declaration block; left-hand-side variable origin=parameter
Illegal statement beginning with non-void expression parsed as
  mu[[i, j]]
Not a legal assignment, sampling, or function statement.  Note that
  * Assignment statements only allow variables (with optional indexes) on the left;
  * Sampling statements allow arbitrary value-denoting expressions on the left.
  * Functions used as statements must be declared to have void returns

 error in 'unknown file name' at line 29, column 13
  -------------------------------------------------
    27:              y[i,j] ~ normal(mu[i,j], sigma);
    28:              
    29:              mu[i,j] = beta1 + r[1,j] + (beta2 + beta3*X[i,j] + r[2,j])*Age[i,j];
                    ^
    30:              
  -------------------------------------------------

PARSER EXPECTED: 